In [ ]:
!pip install lime

In [1]:
def explain_prediction(row, model, vectorizer, scaler, top_n=5):
    """
        Explains the prediction of the model by showing the most influential features (both TF-IDF and readability metrics) for the prediction.

        Parameters:
        ----------
        row : pandas.Series
            A row of the dataframe containing:
              - 'text': the preprocessed text (e.g. from 'content_lemma_nostop')
              - 'word_count': the document's word count
              - 'sentence_count': the number of sentences
              - 'flasch_reading_ease': the readability score
        model : object
            The trained machine learning model.
        vectorizer : object
            The TF-IDF vectorizer used to transform the text.
        scaler : object
            The scaler used for the readability metrics during training.
        top_n : int
            The number of features to display based on their contribution.

        Returns:
        -------
        dict
            A dictionary with the prediction ('label') and the top contributing features ('features').
    """
    #make sure ther eis text in the row
    row['text'] = row['text'] if pd.notnull(row['text']) else ''
    #transform the text using the vectorizer (this gets the first 5000 features)
    tfidf_text = vectorizer.transform([row['text']])
    #get the readability features from the row
    readability_values = np.array([[row['word_count'], row['sentence_count'], row['flesch_reading_ease']]])
    #scale the readability metrics using the same scaler as during training.
    readability_scaled = scaler.transform(readability_values)
    #convert the scaled features into a sparse matrix
    readability_sparse = csr_matrix(readability_scaled)
    #combine TF-IDF features and readability features
    full_features = hstack([tfidf_text, readability_sparse])
    # Predict the label.
    prediction = model.predict(full_features)[0]
    
    #get contribution for each feature
    contributions = full_features.toarray()[0]* model.coef_[0]
    #get indices of features with highest absolute contributions
    top_indices = np.argsort(np.abs(contributions))[-top_n:]
    #get feature names
    tfidf_feature_names = vectorizer.get_feature_names_out()
    readability_feature_names = ['word_count', 'sentence_count', 'flesch_reading_ease']
    all_feature_names = list(tfidf_feature_names) + readability_feature_names
    top_features = [all_feature_names[i] for i in top_indices]

    return {
        "label": prediction,
        "features": top_features
    }

# Create a DataFrame for test data
test_df = pd.DataFrame({
    'text': df.loc[test_indices, 'content_lemma_nostop'].reset_index(drop=True),
    'word_count': df.loc[test_indices, 'word_count'].reset_index(drop=True),
    'sentence_count': df.loc[test_indices, 'sentence_count'].reset_index(drop=True),
    'flesch_reading_ease': df.loc[test_indices, 'flesch_reading_ease'].reset_index(drop=True),
    'true_label': y_test.reset_index(drop=True),
    'predicted_label': y_pred
})

# Row predicted as Real (0)
real_example = test_df[test_df['predicted_label'] == 0].iloc[0]
#print(real_example)

# Row predicted as Fake (1)
fake_example = test_df[test_df['predicted_label'] == 1].iloc[0]

NameError: name 'pd' is not defined

In [ ]:
real_explanation = explain_prediction(real_example, final_model, vectorizer, scaler)
fake_explanation = explain_prediction(fake_example, final_model, vectorizer, scaler)

print("Real Example Prediction:")
print("Text:", real_example['text'])
print("Predicted Label:", real_explanation['label'])
print("Top Features:", real_explanation['features'])

print("\nFake Example Prediction:")
print("Text:", fake_example['text'])
print("Predicted Label:", fake_explanation['label'])
print("Top Features:", fake_explanation['features'])



In [ ]:
def predict_proba(texts):
    """
    This function takes a list of raw text strings, vectorizes them using the
    pre-fitted TF-IDF vectorizer, and returns the probability predictions
    from the trained model.
    """
    return model.predict_proba(vectorizer.transform(texts))

# Create a LIME text explainer
explainer = LimeTextExplainer(class_names=['true', 'fake'])

# Choose an instance from the test set by its index
idx = 0  # you can adjust this index as needed
raw_text = df.loc[test_indices].iloc[idx]['content']

# Use the custom predict_proba function in LIME
exp = explainer.explain_instance(raw_text, predict_proba, num_features=10)
exp.show_in_notebook(text=raw_text)